**Gerar vetor das rotas obtidas via Radares Automotivos**
<br> O script acessa os arquivos gerados via radares automotivos; gera um modelo json para cada rota; carrega os arquivos json; requisita e armazena o valor do jobID de cada um dos arquivos json; requisita e faz o download dos resultados de cada jobID.
<br>
<br> O script divide-se em 3 etapas:
<br>**ETAPA 1:** Define functions para serem usadas em cada arquivo
<br>**ETAPA 2:** Gera os dados jobID em forma de lista (job_id)
<br>**ETAPA 3:** Faz o download dos resultados
<br>**ETAPA 4:** Descompacta e exclui os arquivos zip, mantendo os arquivos GIS e planilha
<br>
<br> obs: Este código foi tratado de uma versão de outra autoria


https://developer.tomtom.com/traffic-stats/traffic-stats-apis/route-analysis
<br> Site com modelo JSON do API, com dica de como deve ficar a estrutura

Dica de site para testar se o arquivo .geojson formado, é um arquivo JSON válido
<br> https://jsonlint.com/

In [ ]:
import datetime
import json
import requests
import pickle
import os
from tqdm import tqdm
import regex as re
import pandas as pd
import zipfile
import shutil

# 1 - DEFINIÇÃO DE FUNÇÕES

### Função: constructor(file)
<br> Função para geração dos arquivos geojson

In [ ]:
# Função para criar cosntructor e modelos Json para cada rota
def constructor(file):
    
    #Lê arquivo do radar atuotomotivo e armazena id da viagem
    rota = json.load(open(r"local"+str(file)))
    viagem_id = rota[0]["viagem_id"]
    print("rota: "+str(file),"\n"
          "viagem_id: "+str(viagem_id),"\n")
    
    # Cria uma pasta para cada viagem
    try:
        foldername = (r"rota_blocos\rota_"+str(viagem_id))
        #print(foldername)
        os.mkdir(foldername)
    except: 
        pass
        #print("Pasta já Existente")
             
    # Criaçao dos arquivos geojson para cada trecho da rota em análise
    # Parte 1: Extração das datas e horários
    data_obj_1 = rota[1]["data_inicio"]
    data_inicio= datetime.datetime.strptime(data_obj_1, "%Y-%m-%dT%H:%M:%SZ")
    data_obj_2 = rota[len(rota)-1]["data_final"]
    data_final= datetime.datetime.strptime(data_obj_2, "%Y-%m-%dT%H:%M:%SZ")
    #print (data_inicio.strftime("%Y-%m-%d %H:%M"))
    #print (data_final.strftime("%Y-%m-%d %H:%M"))
    
    # Parte 2: Escreve o arquivo geojson
    filename = "rota_"+str(viagem_id)+".geojson"
    geojson_file = open(os.path.join(foldername, filename), 'w')
    geojson_file.write('{\n')
    geojson_file.write('"jobName": "rota_'+str(viagem_id)+'",\n')
    geojson_file.write('"distanceUnit":"KILOMETERS",\n')
    geojson_file.write('"routes": [\n{\t"name":"viagem_id_'+str(viagem_id)+'",\n')
    geojson_file.write('\t"start":{"latitude":'+str(rota[1]["lat_inicio"])+',"longitude":'+str(rota[1]["long_inicio"])+'},\n')
    for i in rota[2:len(rota)]:
        geojson_file.write('\t"via":[{"latitude":'+str(i["lat_inicio"])+',"longitude":'+str(i["long_inicio"])+'}],\n')
    geojson_file.write('\t"end":{"latitude":'+str(rota[len(rota)-1]["lat_final"])+',"longitude":'+str(rota[len(rota)-1]["long_final"])+'},\n')
    geojson_file.write('\t"fullTransversal":"false",\n\t"zoneId":"UTC-3",\n\t"probeSource":"ALL"}],\n')
    geojson_file.write('"dateRanges": [{"name":"Via Radares SP", "from": "'+data_inicio.strftime("%Y-%m-%d")+'","to": "'+data_final.strftime("%Y-%m-%d")+'"}],\n')  # Tive que escolher uma data manualmente, a dos radares não funcionou
    geojson_file.write('"timeSets": [{"name":"Via Radares SP", \n"timeGroups":[{"days":["'+data_inicio.strftime("%a").upper()+'"],"times":["'+data_inicio.strftime("%H:%M")+'-'+data_final.strftime("%H:%M")+'"]}]}]\n')  # dia da semana tem que ser em maíusculo e compatível com o dateRange
    geojson_file.write('}')
    geojson_file.close()
    
    return 

### Função: BatchRequestJobID(batch_key)

Requisição (jobID)

In [ ]:
def BatchRequestJobID(batch_key):
    batch_id = []
    global erro_json, erro_count
    for key in tqdm(range(len(batch_list[batch_key]))):
        try:
            r = requests.post(url, json = batch_list[batch_key][key])
            batch_id.append(r.json()['jobId'])
        except:
            print("Erro")
            print(batch_list[batch_key][key])
            erro_json.append(batch_list[batch_key][key])
    return print(batch_id, batch_list[batch_key][key]["jobName"])

In [ ]:
def BatchRequestJobID(batch_key):
    batch_id = {}
    global erro_json, erro_count
    for key in tqdm(range(len(batch_list[batch_key]))):
        try:
            r = requests.post(url, json = batch_list[batch_key][key])
            batch_id[r.json()['jobId']]=batch_list[batch_key][key]["jobName"]
        except:
            print("Erro")
            print(batch_list[batch_key][key])
            erro_json.append(batch_list[batch_key][key])
    return batch_id

### Função: baixar_arquivo(url, endereco)
<br> Função para fazer download de cada arquivo segundo um jobID

In [ ]:
# Função para download dos arquivos
def baixar_arquivo(url, endereco):
    # faz requisição ao servidor
    resposta = requests.get(url)
    if resposta.status_code == requests.codes.OK:
        with open(endereco, 'wb') as novo_arquivo:
            novo_arquivo.write(resposta.content)
        print("Donwload finalizado. Salvo em: {}".format(endereco))
    else:
        resposta.raise_for_status()

# 2 - GERAÇÃO DOS DADOS jobID

In [ ]:
# Cria pasta para armazenamento dos resultados
try:
    foldername_arquivos = r"rota_blocos"
    os.mkdir(foldername_arquivos)
except: 
    print("Pasta já Existente")


In [ ]:
# Cria arquivos .json
arquivos = os.listdir(r"local") # Local dos arquivos gerados via API Radares_SP

viagem_id = []
for file in arquivos:
    viagem_id.append(file.split("_")[1].split(".")[0])
# Aplicação da função Constructor
    try:
        viagem_id_fhand = constructor(file)        
    except: continue

In [ ]:
# Carrega os arquivos .json
geojson = []
viagem_id_erro = []
for i in viagem_id:
    try:
        f = open(foldername_arquivos+r"\rota_"+str(i)+r"\rota_"+str(i)+".geojson")
        geojson.append(json.load(f))
    except:
        viagem_id_erro.append(i)
        

In [ ]:
# Criação do batch
batch_list = []
batch = []
for key in geojson:
    batch.append(key)
batch_list.append(batch.copy())
print(len(batch_list))

In [ ]:
# Requisição jobID

# URL solicitação jobID
url = 'https://api.tomtom.com/traffic/trafficstats/routeanalysis/1?key=<key>'
erro_json = []
for i in tqdm(range(len(batch_list))):
    job_id = BatchRequestJobID(i)

In [ ]:
# job_id é um dicionário com os valores de jobID associados à rota em questão, para descobrir o valor do jobID de uma dada rota
# aplicar o comando a seguir:
#list(job_id.keys())[list(job_id.values()).index("rota_99435878")]

# Salvar job_id via arquivo PKL (savepoint) 
len(job_id)
with open(os.path.join(foldername_arquivos,'jobID.pkl'), 'wb') as f:
    pickle.dump([job_id], f)

# 3 - DOWNLOAD DOS RESULTADOS

In [ ]:
# Carregar job_id
try:
    len(job_id)
except:
    job_id = pd.read_pickle(r"local\jobID.pkl")[0]

In [ ]:
# Lista com valores completos ou rejeitados
try: completed_jobID = pd.read_pickle(r"local\completed_jobID.pkl")[0]
except: completed_jobID = []
    
try: rejected_jobID = pd.read_pickle(r"local\rejected_jobID.pkl")[0]
except: rejected_jobID = []

In [ ]:
# Aplica a função de download, talvez seja necessário rodar mais de uma vez devido a demora nos cálculos dos resultados. Prestar
# atenção na completed_jobID e comparar com a lista job_ID. A lista rejected_jobID apenas contém elementos que já foram
# calculados e rejeitados.

# Outro ponto importante, é que o bloco dá erro algumas vezes quando o status do jobID muda durante o acesso, caso aconteça,
# apenas precisa rodar o bloco novamente, graças a lista completed_jobID, não será preciso refazer todos os downloads.

for i in tqdm(job_id):
    # Pula arquivos que já foram baixados
    if i in completed_jobID or i in rejected_jobID: continue
    
    url = 'https://api.tomtom.com/traffic/trafficstats/status/1/'+i+'?key=<key>'
    r = requests.get(url)    
    # print(r.json(),"\n")
    
    # Verifica o status do jobID, se é possível baixar ou não
    try : 
        r.json()["urls"]
    except:
        print("ERRO","\n"
             "url:",url,"\n")
        if r.json()["jobState"] in ("NEW","CALCULATIONS"):
            print("Resultados ainda sendo calculados", "\n"
                  "jobState:",r.json()["jobState"],"\n")
        elif r.json()["jobState"] == "REJECTED":
            print("Viagem rejeitada \"Could not calculate routes\", confirmar.", "\n"
                  "jobState:",r.json()["jobState"],"\n")
                  #r.json()["messages"])
            rejected_jobID.append(i)
        continue
           
    # Baixa arquivos        
    if __name__ == "__main__":
        BASE_URL = (r.json()['urls'][4])
        OUTPUT_DIR = foldername_arquivos+r"\\"+job_id[i]  # Local à serem salvos os arquivos xlsx das rotas, lembrar de mudar
        nome_arquivo = os.path.join(OUTPUT_DIR, job_id[i]+'.xlsx'.format(i))
        baixar_arquivo(BASE_URL.format(i), nome_arquivo)
    if __name__ == "__main__":
        BASE_URL = (r.json()['urls'][3])
        OUTPUT_DIR = foldername_arquivos+r"\\"+job_id[i] # Local à serem salvos os arquivos kmz das rotas, lembrar de mudar
        nome_arquivo = os.path.join(OUTPUT_DIR, job_id[i]+".zip".format(i))
        baixar_arquivo(BASE_URL.format(i), nome_arquivo)
    if i not in completed_jobID: completed_jobID.append(i)

In [ ]:
# Salvar job_id via arquivo PKL (savepoint) 
with open(os.path.join(foldername_arquivos,'completed_jobID.pkl'), 'wb') as f:
    pickle.dump([completed_jobID], f)

with open(os.path.join(foldername_arquivos,'rejected_jobID.pkl'), 'wb') as f:
    pickle.dump([rejected_jobID], f)

# 4 - Descompactar arquivos shapefiles

In [ ]:
# Descompactar arquivos
for i in completed_jobID:
    # Confere se o arquivo final já existe
    if os.path.isfile(foldername_arquivos+"\\"+str(job_id[i])+"\\viagem_id_"+job_id[i].split("_")[1]+"_1\\route.shp") is True:
        continue
    # Descompacta e exclui o .zip    
    try:
        with zipfile.ZipFile(foldername_arquivos+"\\"+str(job_id[i])+"\\"+str(job_id[i])+".zip") as z:
            z.extractall(path = foldername_arquivos+"\\"+str(job_id[i]),
                         members = z.namelist()[0:4])
        os.remove(foldername_arquivos+"\\"+str(job_id[i])+"\\"+str(job_id[i])+".zip") 
    # Caso nem o arquivo final (.shp) nem o zip seja encontrado, retorna a rota problemática para análise manual
    except: 
        print(job_id[i])
        continue    

In [ ]:
# Exclui pastas das rotas rejeitadas pelo API, ATENÇÃO PARA NÃO EXCLUIR PASTAS NECESSÁRIAS
for i in rejected_jobID:
    #print(foldername_arquivos+r"\rota_"+str(job_id[i]))
    shutil.rmtree(foldername_arquivos+"\\"+str(job_id[i]))